In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import *
from collections import Counter
import seaborn as sns
from tqdm import tqdm
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
IMAGE_DIR = 'image_contest_level_2'
CROP_DIR = 'crop'

In [ ]:
def disp2(img):
    cv2.imwrite('a.png', img)
    return Image('a.png')

def disp(img, txt=None, first=False):
    global index
    if first:
        index = 1
        plt.figure(figsize=(16, 9))
    else:
        index += 1
    plt.subplot(4, 3, index)
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img[:,:,::-1])
    if txt:
        plt.title(txt)

In [ ]:
def plot(index):
    global img, gray, b, eq, bw, m1, m2, r, roi
    img = cv2.imread('%s/%d.png'%(IMAGE_DIR, index))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    eq = cv2.equalizeHist(gray)
    b = cv2.medianBlur(eq, 9)
    
    m, n = img.shape[:2]
    b = cv2.resize(b, (n//4, m//4))

    m1 = cv2.morphologyEx(b, cv2.MORPH_OPEN, np.ones((7, 40)))
    m2 = cv2.morphologyEx(m1, cv2.MORPH_CLOSE, np.ones((3, 3)))
    _, bw = cv2.threshold(m2, 127, 255, cv2.THRESH_BINARY_INV)
    
    bw = cv2.resize(bw, (n, m))

    r = img.copy()
    img2, ctrs, hier = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for ctr in ctrs:
        x, y, w, h = cv2.boundingRect(ctr)
        cv2.rectangle(r, (x, y), (x+w, y+h), (0, 255, 0), 10)
    x, y, w, h = cv2.boundingRect(np.vstack(ctrs))
    
    d = 10
    cv2.rectangle(r, (x-d, y-d), (x+w+d, y+h+d), (0, 0, 255), 10)
    roi = img[max(0, y-d):min(m, y+h+d),max(0, x-d):min(n, x+w+d)]

    disp(img, 'raw img', 1)
    disp(eq, 'eq')
    disp(b, 'blur')
    disp(m2, 'm')
    disp(r, 'rect')
    disp(roi, 'roi')
    
    for i, ctr in enumerate(ctrs):
        x, y, w, h = cv2.boundingRect(ctr)
        roi = img[max(0, y-d):min(m, y+h+d),max(0, x-d):min(n, x+w+d)]
        disp(roi, 's%d'%i)
    
# plot(53698)

In [ ]:
df = pd.read_csv('size_test.csv')
df

In [ ]:
(df['r'] > 3).sum()

In [ ]:
pd.plotting.scatter_matrix(df, alpha=0.1, figsize=(14,8), diagonal='kde');

In [ ]:
df = pd.read_csv('size.csv')
df

In [ ]:
Counter(df['m'])

In [ ]:
Counter(df['n'])

In [ ]:
(df['r'] == 2).sum()

In [ ]:
(df['r'] == 3).sum()

In [ ]:
(df['r'] > 3).sum()

In [ ]:
pd.plotting.scatter_matrix(df, alpha=0.1, figsize=(14,8), diagonal='kde');

In [ ]:
plot(99982)

# 公式有两个和三个的差别

In [ ]:
df.plot.scatter('h', 'w')

# 出现的位置 (x, y) 与框的宽度和高度 (w, h) 没有关系

In [ ]:
df.plot.scatter('h', 'x')
df.plot.scatter('h', 'y')

df.plot.scatter('w', 'x')
df.plot.scatter('w', 'y')

# 公式数量与高度正相关

In [ ]:
df.plot.scatter('h', 'r')